In [1]:
# import dependencies

from splinter import Browser
from bs4 import BeautifulSoup as bs
#import time
#import pymysql
#pymysql.install_as_MySQLdb()


#import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import select
import mysql_scr


from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func



In [2]:
init_string = (f"root:{mysql_scr.pw}@localhost:{mysql_scr.dbport}")

engine = create_engine(f"mysql://{init_string}/tree_db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['nyc_tree', 'tree_species']

In [3]:
#initialize base
Tree = Base.classes.tree_species
session = Session(engine)
Tree_species = session.query(Tree.species_nm).all()

#declare tree_list variable and append from database table
tree_list = []

for tree in Tree_species:
    tree_list.append(tree.species_nm)
    
print(tree_list)

#text = session.query(func.lower(func.replace(Tree.species_nm,' ','')).label("species_nm"))
#for i in text:
    #print(i.species_nm)

[]


In [ ]:
#Create webkey (will be replaced) for looking up to website
trees  = [x.replace(' ','').lower() for x in tree_list]

for i in trees:
    print (i)

In [6]:
#Open brower connection

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [8]:
#declare url and parser
url = "http://leafsnap.com/species/"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [11]:
#Get webscraping!
#Get div data related to the first species tab
ne_species = soup.find('div', id='species-1')

In [18]:
#find all tr rows to loop through the data
species = ne_species.table.tbody.find_all('tr')

for td in species:
    img = td.find_all('img')
    name = td.find_all('td')
    link = td.find_all('a')
    qry_name = name[3].text.replace(" ", "").lower()
    
    #check to see if web tree is in sql tree
    if qry_name in trees:
        image_link = img[0]['src']
        web_name = name[3].text
        print(image_link)
        print(name[3].text)
        print(link[0]['href'])
        
        ## update sql table
        session.query(Tree.img_loc)\
                .filter(func.lower(func.replace(Tree.species_nm,' ','')) == qry_name)
                \.update({"img_loc":image_link, "web_common_nm":web_name}, synchronize_session='fetch')
        session.commit()

        ## go to tree data page
        browser.click_link_by_partial_href(link[0]['href'])
        
        #get new webpage html and parse
        html_species = browser.html
        soup_species = bs(html_species, "html.parser")
        
        tree = soup_species.find_all('dd')

        habitat = tree[0].text
        growth = tree[1].text
        bloom = tree[2].text
        longevity = tree[3].text
        
        session.query(Tree.habitat, Tree.growth_habit, Tree.bloom_time, Tree.longevity )\
                .filter(func.lower(func.replace(Tree.species_nm,' ','')) == qry_name)\
                .update({"habitat":habitat, "growth_habit": growth, "bloom_time": bloom, "longevity": longevity}, synchronize_session='fetch')

        session.commit()
        print(habitat, growth, bloom, longevity)
 
        browser.back()

        

http://api.leafsnap.com/v1/species/Gleditsia%20triacanthos/images/LTV-EJF-00422_crop.jpg?crop=-158,64,1221,1444&h=150&w=150
Honeylocust
Gleditsia triacanthos/
Woodland edges and open areas. Deciduous tree growing to 30 m tall, with open crown. Late spring. Up to 100 years.
http://api.leafsnap.com/v1/species/Platanus%20acerifolia/images/LTV-RBD-06693_crop.jpg?crop=11,89,1063,1140&h=150&w=150
London Plane  Tree
Platanus acerifolia/
Planted as an ornamental, growing best in full sun. Deciduous tree, growing 21-30 m tall or higher. Late spring. Long-lived.
